In [49]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score


import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

2024-07-24 16:03:26.219746: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 16:03:26.219882: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 16:03:26.380330: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [3]:
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
train_df.shape, test_df.shape

((7613, 5), (3263, 4))

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [8]:
count_vectorizer = feature_extraction.text.CountVectorizer()

train_vectors = count_vectorizer.fit_transform(train_df["text"])
test_vectors = count_vectorizer.transform(test_df["text"])

In [9]:
train_vectors.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [18]:
feature_names = count_vectorizer.get_feature_names_out()
print(feature_names)

['00' '000' '0000' ... 'ûónegligence' 'ûótech' 'ûówe']


In [19]:
clf = linear_model.RidgeClassifier()

In [20]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.59453669, 0.5642787 , 0.64082434])

In [21]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier()

In [22]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [23]:
sample_submission["target"] = clf.predict(test_vectors)

In [24]:
sample_submission.to_csv("submission.csv", index=False)

In [40]:
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the documents
tfidf_matrix = tfidf_vectorizer.fit_transform(train_df["text"])

# Print the TF-IDF matrix
print(tfidf_matrix.toarray())

# Get feature names (words)
print(tfidf_vectorizer.get_feature_names_out())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
['00' '000' '0000' ... 'ûónegligence' 'ûótech' 'ûówe']


In [41]:
tfidf_matrix_test = tfidf_vectorizer.transform(test_df["text"])

In [42]:
clf = linear_model.RidgeClassifier()

In [43]:
scores = model_selection.cross_val_score(clf, tfidf_matrix, train_df["target"], cv=3, scoring="f1")
scores

array([0.63366337, 0.6122449 , 0.68407835])

In [44]:
clf.fit(tfidf_matrix, train_df["target"])
sample_submission["target"] = clf.predict(tfidf_matrix_test)
sample_submission.to_csv("submission_2.csv", index=False)

In [45]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer



# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(train_df["text"])
tfidf_matrix_test = tfidf_vectorizer.transform(test_df["text"])


# Initialize the Truncated SVD (LSA)
lsa = TruncatedSVD(n_components=2)  # Number of components (topics)
lsa_matrix = lsa.fit_transform(tfidf_matrix)
lsa_matrix_test = lsa.fit_transform(tfidf_matrix_test)


# Print the LSA matrix
print(lsa_matrix)

# Print the terms most associated with each topic
terms = tfidf_vectorizer.get_feature_names_out()
for i, component in enumerate(lsa.components_):
    terms_in_topic = [terms[i] for i in component.argsort()[:-5 - 1:-1]]
    print(f"Topic {i}: {terms_in_topic}")


[[ 0.1314479  -0.05721855]
 [ 0.02381474  0.00888556]
 [ 0.11474927 -0.02732399]
 ...
 [ 0.08035902  0.07875078]
 [ 0.06277129 -0.00281224]
 [ 0.17909042  0.1759709 ]]
Topic 0: ['co', 'http', 'the', 'to', 'in']
Topic 1: ['http', 'co', 'via', 'new', 'û_']


In [46]:
clf = linear_model.RidgeClassifier()
scores = model_selection.cross_val_score(clf, lsa_matrix, train_df["target"], cv=3, scoring="f1")
scores

array([0.53620955, 0.5247225 , 0.62082139])

In [47]:
clf.fit(lsa_matrix, train_df["target"])
sample_submission["target"] = clf.predict(lsa_matrix_test)
sample_submission.to_csv("submission_2.csv", index=False)

In [50]:
# Parameters
max_words = 1000  # Maximum number of words in the vocabulary
max_len = 10      # Maximum length of sequences

# Tokenizer for text data
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_df['text'])
sequences = tokenizer.texts_to_sequences(train_df['text'])
X = pad_sequences(sequences, maxlen=max_len)

# Convert target to numpy array
y = np.array(train_df['target'])

def create_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_words, output_dim=50, input_length=max_len))
    model.add(LSTM(64))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

kf = KFold(n_splits=3, shuffle=True, random_state=42)

f1_scores = []

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    model = create_model()
    model.fit(X_train, y_train, epochs=5, batch_size=2, verbose=0)
    
    # Evaluate model on validation set
    y_pred = (model.predict(X_val) > 0.5).astype(int)
    f1 = f1_score(y_val, y_pred)
    f1_scores.append(f1)

print("F1 Scores:", f1_scores)
print("Mean F1 Score:", np.mean(f1_scores))

# Now, predict on the test set
test_sequences = tokenizer.texts_to_sequences(test_df['text'])
X_test = pad_sequences(test_sequences, maxlen=max_len)

# Predict probabilities and convert to binary predictions
y_test_pred_prob = model.predict(X_test)
y_test_pred = (y_test_pred_prob > 0.5).astype(int)

# If you have true labels for the test set and want to evaluate:
if 'target' in test_df.columns:
    y_test_true = np.array(test_df['target'])
    f1_test = f1_score(y_test_true, y_test_pred)
    print("Test F1 Score:", f1_test)

# If only predictions are needed, print or store them
print("Test Predictions:", y_test_pred)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
F1 Scores: [0.7070124879923151, 0.7136563876651981, 0.7154394299287412]
Mean F1 Score: 0.7120361018620848
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Test Predictions: [[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]


In [51]:
sample_submission["target"] = y_test_pred
sample_submission.to_csv("submission_3.csv", index=False)